In [18]:
from smb.SMBConnection import SMBConnection
import os
from pathlib import Path
import pickle
from datetime import datetime

share_name = "13810008311_Home1"
remote_dir = "/Backup/iPhoneXR_undocumented/"
nodel_dir = "/Backup/来自wp的相册/"

首先，获得所有影像文件，包括确定保留的文件HEIC，MOV格式的，和可能删除的重复的文件JPG，MOV格式的。将影像文件list存入.lst格式的文件中，方便后续处理。否则，反复在SMB中读取影像文件信息，处理速度会非常慢。

In [15]:
def smbwalk(conn, shareddevice, top = u'/'):
    dirs , nondirs = [], []

    if not isinstance(conn, SMBConnection):
        raise TypeError("SMBConnection required")


    names = conn.listPath(shareddevice, top)

    for name in names:
        if name.isDirectory:
            if name.filename not in [u'.', u'..']:
                dirs.append(name.filename)
        else:
            nondirs.append(name.filename)

    yield top, dirs, nondirs

    for name in dirs:
        new_path = os.path.join(top, name)
        for x in smbwalk(conn, shareddevice, new_path):
            yield x

def findAllFile(conn, share_name ,top):
    for root, ds, fs in smbwalk(conn, share_name ,top):
        for f in fs:
            fullname = os.path.join(root, f)
            yield fullname

conn = SMBConnection("13810008311", "y6c3lh",'','')
assert conn.connect('192.168.31.28',445)

In [ ]:
nodel_file = []
q_file = []
          
for f in findAllFile(conn, share_name ,top= nodel_dir):
    nodel_file.append(f)
for f in findAllFile(conn, share_name ,top= remote_dir):
    q_file.append(f)
    


In [8]:
def write_list(list_name,filename):
    # store list in binary file so 'wb' mode
    with open(filename, 'wb') as fp:
        pickle.dump(list_name, fp)
        print('Done writing list into a binary file')
        
# Read list to memory
def read_list(filename):
    # for reading also binary mode is important
    with open(filename, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list
    
write_list(nodel_file, 'nodel.lst')
write_list(q_file, 'q_file.lst')

Done writing list into a binary file
Done writing list into a binary file


根据文件名，确定要删除的影响文件，放入del_file.lst。

In [10]:
q_file_r = read_list( 'q_file.lst')
nodel_r = read_list( 'nodel.lst')
delfile = []
reserve_file = []
for f in q_file_r:
    obj_file = Path(f).stem
    for f2 in nodel_r:
        delornot = Path(f2).stem
        if obj_file == delornot:
            delfile.append(f)
            reserve_file.append(f2)
            # print(f+' should be del')

write_list(delfile, 'del_file.lst')
write_list(reserve_file, 'reserve_file.lst')

Done writing list into a binary file
Done writing list into a binary file


In [11]:
delfile = read_list( 'del_file.lst')
reserve_file = read_list( 'reserve_file.lst')

但是，仅仅根据文件名确定删除对象，不是很保险，希望加入photo taken time的判断。
通过SMB进行exif信息的提取，是不合适的，因为只有create_time，HEIC文件的create_time确实等于拍摄时间，而JPG的却不是。所以，这一条路不通。
后续还是，通过将所有文件copy到本地PC进行处理。不再通过SMB进行。

In [31]:
for idx, f in enumerate(delfile):
    attr1 = conn.getAttributes(share_name, f)
    attr2 = conn.getAttributes(share_name, reserve_file[idx])
    # print(f + ' shot time: ' )
    # print(datetime.fromtimestamp(attr1.create_time))
    # print(reserve_file[idx] + ' shot time: ' )
    # print(datetime.fromtimestamp(attr2.create_time))
    if attr1.create_time -attr2.create_time > 24*3600 :
        print('NO same shot time')

    if idx >= 1966:
        break



NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same shot time
NO same sh